In [30]:
# import libs

from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

from scipy.spatial import distance_matrix

import datetime
import numpy as np
import numpy.random as nr
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd

import random
import copy
import time
import itertools


In [31]:
# init parameters

NO_TOWNS = 10 # number of towns (town numbering is from [0, NO_TOWNS-1])
NO_HOURS = 24 # number of hour intervals (e.g. 0000 - 0100, 0100 - 0200)
NO_AGENTS = 1
STARTING_TOWN = 0 # set starting town
TRAVEL_SPEED = 1 # km/min
nr.seed(69) # set fixed seed for rng



In [32]:
# function for creating town coordinates
def create_towns(NO_TOWNS):
    """generate random x and y coordinates for each town"""

    towns_x = nr.randint(0, 100, NO_TOWNS) # x coordinates for NO_TOWNS number of towns
    towns_y = nr.randint(0, 100, NO_TOWNS) # y coordinates for NO_TOWNS number of towns
    
    return towns_x, towns_y


towns_x, towns_y = create_towns(NO_TOWNS) # generate coordinates for each town

start_town_x = towns_x[STARTING_TOWN]
start_town_y = towns_y[STARTING_TOWN]

In [45]:
def create_distance_matrix(towns_x, towns_y):
    """create a matrix of pairwise distances between all towns"""
    towns = list(range(NO_TOWNS))
    df = pd.DataFrame(np.array([towns_x, towns_y]).transpose(), columns=['x', 'y'], index=towns)
    distance_df = pd.DataFrame(distance_matrix(df.values, df.values), index=df.index, columns=df.index)
    return np.round_(distance_df).astype(int)

def create_travel_time_matrix(distance_matrix, travel_speed):
    """create a matrix of pairwise travel times between all towns in minutes"""
    
    return distance_matrix/travel_speed
    
    
def create_waiting_time_matrix(method='calculate', time=7):
    """
    create a matrix of waiting times at each town
    method = ['calculate', 'update']
    """
    
    if method == 'calculate':
        
        def time_fn(time):
            return (t-13)**2 + 60
        
        matrix = np.array([time_fn(time), time_fn(time), time_fn(time), time_fn(time), time_fn(time), 
                          time_fn(time), time_fn(time), time_fn(time), time_fn(time), time_fn(time), ]) # each time function can be different
        
    elif method == 'update':
        def time_update():
            conn = sqlite3.connect('test.db') # connect to real_time waiting time database
            df = pd.DataFrame(conn.execute('SELECT * FROM waiting_times').fetchall(), columns=['town_id', 'waiting_time'])
            return df.to_numpy()
        
        matrix = time_update()
        
    return matrix.astype(int)





In [46]:
# Creating callback function for travelling distance cost

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    
    return data['dist_matrix'][from_node][to_node]

def travel_time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    
    return data['travel_time_matrix'][from_node][to_node]
    

# Creating callback function for waiting time cost
def waiting_time_callback(to_index):
    """Returns the waiting time at a node depending on the time interval (lower bound hour of the interval)"""
    # Convert from routing variable index to time matrix NodeIndex
    to_node = manager.IndexToNode(to_index)
#     time = travel_time_dimension.CumulVar(to_node)
#     print(type(time))
    return data['waiting_time_matrix'][to_node]


def cost_callback(from_index, to_index):
    """Returns the total cost"""
    return distance_callback(from_index, to_index) + 4*waiting_time_callback(to_index)


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['dist_matrix'] = dist_matrix
    data['travel_time_matrix'] = travel_time_matrix
    data['waiting_time_matrix'] = waiting_time_matrix
    data['num_vehicles'] = NO_AGENTS
    data['depot'] = STARTING_TOWN
    
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}\n')
    
    max_route_distance = 0
    max_waiting_time = 0
    routes = []
    route_distances = []
    route_waiting_times = []
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        
        route = []
        route_distance = 0
        route_waiting_time = 0
        
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            route.append(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            print(type(previous_index), type(index), type(vehicle_id))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            route_waiting_time += data['waiting_time_matrix'][manager.IndexToNode(index)]
        
        routes.append(route)
        route_distances.append(route_distance)
        route_waiting_times.append(route_waiting_time)
        
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Total Waiting Time along route: {} minutes\n'.format(route_waiting_time)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
        max_waiting_time = max(route_waiting_time, max_waiting_time)
        
    print('Maximum of the route distances: {}m'.format(max_route_distance))
    print('Maximum of the waiting times: {} minutes'.format(max_waiting_time))
    
    return routes, route_distances, route_waiting_times


In [54]:
"""Entry point of the program."""

walltime = time.time()
cputime = time.process_time()

# Instantiate the data problem.
dist_matrix = create_distance_matrix(towns_x, towns_y)
travel_time_matrix = create_travel_time_matrix(dist_matrix, TRAVEL_SPEED)
waiting_time_matrix = create_waiting_time_matrix('update') # 7 = 7am

data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['dist_matrix']),
                                       data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Create and register a transit callback.
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Create and register a waiting time callback.
wait_callback_index = routing.RegisterUnaryTransitCallback(waiting_time_callback)

# Add Time constraint.
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    1000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    'Distance')

# Add Waiting Time constraint
routing.AddDimension(
    wait_callback_index,
    0,
    200,
    True,
    'Waiting Time')

distance_dimension = routing.GetDimensionOrDie('Distance')
distance_dimension.SetGlobalSpanCostCoefficient(100)

waiting_time_dimension = routing.GetDimensionOrDie('Waiting Time')
waiting_time_dimension.SetGlobalSpanCostCoefficient(100)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Setting time limit on solver
search_parameters.time_limit.seconds = 30

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

self_walltime = time.time()-walltime
self_cputime = time.process_time()-cputime

print('Wall time', self_walltime)
print('CPU time', self_cputime)

# Print solution on console.
if solution:
    routes, route_distances, route_waiting_times = print_solution(data, manager, routing, solution)
else:
    print('No solution found !')


Wall time 0.011918306350708008
CPU time 0.015625
Objective: 0

<class 'int'> <class 'int'> <class 'int'>
<class 'int'> <class 'int'> <class 'int'>
<class 'int'> <class 'int'> <class 'int'>
<class 'int'> <class 'int'> <class 'int'>
<class 'int'> <class 'int'> <class 'int'>
<class 'int'> <class 'int'> <class 'int'>
<class 'int'> <class 'int'> <class 'int'>
<class 'int'> <class 'int'> <class 'int'>
<class 'int'> <class 'int'> <class 'int'>
<class 'int'> <class 'int'> <class 'int'>
Route for vehicle 0:
 0 ->  9 ->  8 ->  7 ->  6 ->  5 ->  4 ->  3 ->  2 ->  1 -> 0
Distance of the route: 509m
Total Waiting Time along route: [ 45 158] minutes



ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [53]:
data = {'town_id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        'waiting_time': [7, 7, 12, 23, 14, 21, 32, 6, 16, 20]}

df = pd.DataFrame(data, columns=['town_id', 'waiting_time'])


In [27]:
import sqlite3
from sqlite3 import Error

conn = sqlite3.connect('test.db')
conn.execute('DROP TABLE IF EXISTS waiting_times')

In [28]:
df.to_sql('waiting_times', conn, index=False)

10

In [29]:
conn.execute('SELECT * FROM waiting_times').fetchall()

[(0, 7),
 (1, 7),
 (2, 12),
 (3, 23),
 (4, 14),
 (5, 21),
 (6, 32),
 (7, 6),
 (8, 16),
 (9, 20)]